In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Analysis of pricing optimization on CDM Pricing Data

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/pricing_optimization/pricing-optimization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fworkbench%2Fpricing_optimization%2Fpricing-optimization.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/pricing_optimization/pricing-optimization.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/pricing_optimization/pricing-optimization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


## Overview
<a name="section-1"></a>

This notebook demonstrates analysis of pricing optimization on [CDM Pricing Data](https://github.com/trifacta/trifacta-google-cloud/tree/main/design-pattern-pricing-optimization) and automating the workflow using Vertex AI Workbench managed notebooks.

**Note**: This notebook file is developed to run in a [Vertex AI Workbench managed notebooks](https://console.cloud.google.com/vertex-ai/workbench/list/managed) instance using the Python (Local) kernel. Some components of this notebook may not work in other notebook environments.

Learn more about [Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/introduction) and Learn more about [BigQuery ML](https://cloud.google.com/vertex-ai/docs/beginner/bqml#machine_learning_directly_in).

### Objective

The objective of this notebook is to build a pricing optimization model using BigQuery ML.

This tutorial uses the following Google Cloud ML services and resources:

- Google Cloud Storage
- BigQuery


The steps performed include:

- Load the required dataset from a Cloud Storage bucket.
- Analyze the fields present in the dataset.
- Process the data to build a model.
- Build a BigQuery ML forecast model on the processed data.
- Get forecasted values from the BigQuery ML model.
- Interpret the forecasts to identify the best prices.
- Clean up.


### Dataset
<a name="section-3"></a>

The dataset used in this notebook is a part of the [CDM Pricing dataset](https://github.com/trifacta/trifacta-google-cloud/blob/main/design-pattern-pricing-optimization/CDM_Pricing_large_table.csv), which consists of product sales information on specified dates.

### Costs
<a name="section-4"></a>

This tutorial uses the following billable components of Google Cloud:

- Vertex AI
- BigQuery
- Cloud Storage


Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
! pip3 install --quiet --upgrade pandas-gbq 'google-cloud-bigquery[bqstorage,pandas]' seaborn fsspec gcsfs


### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project. Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# set the project id
! gcloud config set project {PROJECT_ID}

### Import the required libraries and define constants


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from google.cloud import bigquery
from google.cloud.bigquery import Client

#### Set the BigQuery dataset ID and table ID

In [ ]:
DATASET = "pricing_optimization_unique"  # set the BigQuery dataset-id
TRAINING_DATA_TABLE = (
    "training_data_table"  # set the BigQuery table-id to store the training data
)

## Create a BigQuery dataset
<a name="section-5"></a>


If you're using ***Vertex AI Workbench managed notebooks instance***, every cell which starts with "#@bigquery" is a SQL Query. If you're using a Vertex AI Workbench user-managed notebooks instance or Colab, it's a markdown cell.

#@bigquery
-- create a dataset in BigQuery

CREATE SCHEMA [your-dataset-id]
OPTIONS(
  location="us"
  )

In [ ]:
# Construct a BigQuery client object.
client = Client(project=PROJECT_ID)

In [ ]:
query = """
CREATE SCHEMA {DATASET}
OPTIONS(
  location="us"
  )
""".format(
    DATASET=DATASET
)
query_job = client.query(query)
print(query_job.result())

## Load the BigQuery table from cloud storage
<a name="section-6"></a>


In [ ]:
table_id_name = f"{PROJECT_ID}.{DATASET}.data"

In [ ]:
table_id = "data"

In [ ]:
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
uri = "gs://cloud-samples-data/ai-platform-unified/datasets/tabular/cdm_pricing_large_table.csv"

load_job = client.load_table_from_uri(
    uri, table_id_name, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id_name)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Build a forecast model on this data to determine the best price for a product. For this type of model, you need only a few fields: the sales and price related ones. For the current exercise, focus is on the following fields:

- `Product_ID`
- `Customer_Hierarchy`
- `Fiscal_Date`
- `List_Price_Converged`
- `Invoiced_quantity_in_Pieces`
- `Net_Sales`

## Data Analysis


First, explore the data and it's distributions.

#### Select the required columns from the dataframe.

In [ ]:
id_col = "Product_ID"
date_col = "Fiscal_Date"
categ_cols = ["Customer_Hierarchy"]
num_cols = ["List_Price_Converged", "Invoiced_quantity_in_Pieces", "Net_Sales"]
required_columns = [id_col] + [date_col] + categ_cols + num_cols
required_columns

Create a view to extract only the required columns.

In [ ]:
query = """
    CREATE OR REPLACE TABLE {DATASET}.required_columns AS
    ( SELECT Product_ID,Fiscal_Date,Customer_Hierarchy,List_Price_Converged,Invoiced_quantity_in_Pieces,Net_Sales FROM `{DATASET}.{table_id}` )
    
""".format(
    DATASET=DATASET, table_id=table_id
)

query_job = client.query(query)  # Make an API request.
print(query_job.result())

Inspect data in the `required_columns` view

In [ ]:
query = """
    SELECT * FROM {DATASET}.required_columns 
    
""".format(
    DATASET=DATASET
)

query_job = client.query(query)  # Make an API request.
print(query_job.result())

In [ ]:
query_job.to_dataframe()

#### Check the column types and null values in the dataframe.

In [ ]:
query_job.to_dataframe().info()

This data description reveals that there are no null values in the data. Also, the field `Fiscal_Date` which is a date field is loaded as an object type. 

#### Change the type of the date field to datetime.

Change the `Fiscal_Date` data type from datetime to date and store the resulting data in a view.

In [ ]:
query = """
CREATE OR REPLACE VIEW {DATASET}.required_columns_final AS
(
SELECT Product_ID,Customer_Hierarchy,List_Price_Converged,Invoiced_quantity_in_Pieces,Net_Sales,CAST(DATE(Fiscal_Date) AS DATE) AS Fiscal_Date FROM {DATASET}.required_columns    
)
""".format(
    DATASET=DATASET
)

query_job = client.query(query)  # Make an API request.
print(query_job.result())

Inspect data in the `required_columns_final` view

In [ ]:
query = """
SELECT * FROM {DATASET}.required_columns_final

""".format(
    DATASET=DATASET
)

query_job = client.query(query)  # Make an API request.
print(query_job.result())

In [ ]:
required_columns_final_df = query_job.to_dataframe()

In [ ]:
required_columns_final_df

#### Plot the distributions for the categorical fields.

In [ ]:
for i in categ_cols:
    required_columns_final_df[i].value_counts(normalize=True).plot(kind="bar")
    plt.title(i)
    plt.show()

#### Plot the distributions for the numerical fields.

In [ ]:
for i in num_cols:
    _, ax = plt.subplots(1, 2, figsize=(10, 4))
    required_columns_final_df[i].plot(kind="box", ax=ax[0])
    required_columns_final_df[i].plot(kind="hist", ax=ax[1])
    ax[0].set_title(i + "-Boxplot")
    ax[1].set_title(i + "-Histogram")
    plt.show()

#### Check the maximum date and minimum date in Fiscal_Date column.

In [ ]:
print(required_columns_final_df["Fiscal_Date"].max())
print(required_columns_final_df["Fiscal_Date"].min())

#### Check the product distribution across each category.

In [ ]:
query = """
SELECT Customer_Hierarchy,COUNT(*) as count FROM (SELECT Customer_Hierarchy,Product_ID FROM {DATASET}.required_columns_final GROUP BY Customer_Hierarchy,Product_ID) GROUP BY  Customer_Hierarchy
""".format(
    DATASET=DATASET
)
query_job = client.query(query)
print(query_job.result())

In [ ]:
query_job.to_dataframe()

#### Check the percentage changes in the orders based on the percentage changes in the price.

Execute the following steps to check percentage changes in the orders based on the percentage changes in their price.

**Step 1**: First, generate a table that records each instance of a price change for a product. Include details such as the number of items ordered at each price point and the total net sales associated with that price.

In [ ]:
query = """
create table {DATASET}.price_changes as (
select
       product_id,
       list_price_converged,
       total_ordered_pieces,
       total_net_sales,
       first_price_date,
       lag(list_price_converged) over(partition by product_id order by first_price_date asc) as previous_list,
       lag(total_ordered_pieces) over(partition by product_id order by first_price_date asc) as previous_total_ordered_pieces,
       lag(total_net_sales) over(partition by product_id order by first_price_date asc) as previous_total_net_sales,
       lag(first_price_date) over(partition by product_id order by first_price_date asc) as previous_first_price_date,
       
       
       from (
           select
               product_id,list_price_converged,sum(invoiced_quantity_in_pieces) as total_ordered_pieces, sum(net_sales) as total_net_sales, min(fiscal_date) as first_price_date
           from `{DATASET}.required_columns_final` AS cdm_pricing
           group by 1,2
           order by 1, 2 asc
       )
);

""".format(
    DATASET=DATASET
)
query_job = client.query(query)
print(query_job.result())

In [ ]:
query = """
select * from {DATASET}.price_changes order by product_id, first_price_date 
""".format(
    DATASET=DATASET
)
query_job = client.query(query)
print(query_job.result())

In [ ]:
df_price_changes = query_job.to_dataframe()

In [ ]:
df_price_changes

**Step 2**. Next, with the temporary table in place, you can calculate the price change across SKUs

Ex: (`previous_list`-`list_price_converged`)/nullif(`previous_list`,0)*100

**Step 3**. Next, you can calculate the `total_ordered_pieces` change across SKUs
(`total_ordered_pieces`-`previous_total_ordered_pieces`)/nullif(`previous_total_ordered_pieces`,0)*100 

In [ ]:
query = """
select *,(list_price_converged-previous_list)/nullif(previous_list,0)*100 as price_change_perc,(total_ordered_pieces-previous_total_ordered_pieces)/nullif(previous_total_ordered_pieces,0)*100 as order_change_perc  from `{DATASET}.price_changes`
""".format(
    DATASET=DATASET
)
query_job = client.query(query)
print(query_job.result())

Now you have a dataframe(`df_for_plot`) which has `price_change_perc`, `order_change_perc`fields

In [ ]:
df_for_plot = query_job.to_dataframe()

In [ ]:
# sort values chronologically
df_for_plot.sort_values(by=["product_id", "first_price_date"], inplace=True)
df_for_plot.reset_index(drop=True, inplace=True)

In [ ]:
df_for_plot

Finally, analyze the relationship between each price change and the total number of items ordered.

In [ ]:
# plot a scatterplot to visualize the changes
sns.scatterplot(
    x="price_change_perc",
    y="order_change_perc",
    data=df_for_plot,
    hue="product_id",
    legend=False,
)
plt.title("Percentage of change in price vs order")
plt.show()

For most of the products, the percentage change in orders are high where the percentage changes in the prices are low. This suggests that too much change in the prices can affect the number of orders. 

**Note**: The data contains outliers, with percentage changes exceeding 800. In the current exercise, you can refrain from manually handling outliers, as the BigQuery ML timeseries model is designed to manage outliers automatically.

## Preprocess the data for training

#### Check which `Product_ID`'s  have the maximum orders.

Create a view that stores the number of orders for each product categorized by `Customer_Hierarchy`.

In [ ]:
query = """
CREATE OR REPLACE VIEW {DATASET}.total_orders AS
(
SELECT Customer_Hierarchy,Product_ID,SUM(Invoiced_quantity_in_Pieces) AS Invoiced_quantity_in_Pieces FROM {DATASET}.required_columns_final GROUP BY Customer_Hierarchy,Product_ID

)
""".format(
    DATASET=DATASET
)
query_job = client.query(query)
print(query_job.result())

In [ ]:
query = """
SELECT * FROM {DATASET}.total_orders""".format(
    DATASET=DATASET
)
query_job = client.query(query)
print(query_job.result())

In [ ]:
# sort values chronologically
df_total_orders = query_job.to_dataframe()
df_total_orders.sort_values(by=["Product_ID"], inplace=True)
df_total_orders.reset_index(drop=True, inplace=True)

In [ ]:
df_total_orders

#### Select top products in each Customer_Hierarchy

Below is an example demonstrating how to identify the top products within each `Customer_Hierarchy`.

Example:
Assume that the `total_orders` view is

<table>
    <tr>
        <th>
            Customer_Hierarchy
        </th>
        <th> 
            Invoiced_quantity_in_Pieces
        </th>
        <th> 
            Product_ID
        </th>
    </tr>    
    <tr> 
        <td>Food</td>                 
        <td>200</td>                       
        <td>1</td> 
    </tr>
    <tr> 
        <td>Paper</td>                 
        <td>100</td>                       
        <td>2</td> 
    </tr>
    <tr> 
        <td>Food</td>                 
        <td>300</td>                       
        <td>3</td> 
    </tr>
    <tr> 
        <td>Paper</td>                 
        <td>400</td>                       
        <td>4</td> 
    </tr>
   
</table>    

After partitioning the `total_orders` view by `Customer_Hierarchy` and ordering by `Invoiced_quantity_in_Pieces` in descending order, it becomes

<table>
    <tr>
        <th>
            Customer_Hierarchy
        </th>
        <th> 
            Invoiced_quantity_in_Pieces
        </th>
        <th> 
            Product_ID
        </th>
    </tr>    
    <tr> 
        <td>Food</td>                 
        <td>300</td>                       
        <td>3</td> 
    </tr>
    <tr> 
        <td>Food</td>                 
        <td>200</td>                       
        <td>1</td> 
    </tr>
    <tr> 
        <td>Paper</td>                 
        <td>100</td>                       
        <td>2</td> 
    </tr>
    <tr> 
        <td>Paper</td>                 
        <td>400</td>                       
        <td>4</td> 
    </tr>
</table>   

Now, for every `Customer_Hierarchy`, `Invoiced_quantity_in_Pieces` is arranged in descending order. Then, apply the `ROW_NUMBER` function to the table.

<table>
    <tr>
        <th>
            Customer_Hierarchy
        </th>
        <th> 
            Invoiced_quantity_in_Pieces
        </th>
        <th> 
            Product_ID
        </th>
        <th>
            rowNumber
        </th>    
    </tr>    
    <tr> 
        <td>Food</td>                 
        <td>300</td>                       
        <td>3</td>
        <td>1</td>
    </tr>
    <tr> 
        <td>Food</td>                 
        <td>200</td>                       
        <td>1</td>
        <td>2</td>
    </tr>
    <tr> 
        <td>Paper</td>                 
        <td>100</td>                       
        <td>2</td> 
        <td>1 </td>
    </tr>
    <tr> 
        <td>Paper</td>                 
        <td>400</td>                       
        <td>4</td> 
        <td>2</td>
    </tr>
</table>   

(Each unique `Customer_Hierarchy` is assigned a number starting from 1.)


In [ ]:
query = """
SELECT 
  *,
  ROW_NUMBER() OVER(PARTITION BY Customer_Hierarchy ORDER BY Invoiced_quantity_in_Pieces DESC) rowNumber
  FROM {DATASET}.total_orders
""".format(
    DATASET=DATASET
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

As observed, when arranging by `Customer_Hierarchy`, `Invoiced_quantity_in_Pieces` is in descending order, with rowNumber starting from 1

In [ ]:
query_job.to_dataframe().loc[query_job.to_dataframe()["Customer_Hierarchy"] == "Paper"]

Select the row where `Invoiced_quantity_in_Pieces` is the highest within each Customer_Hierarchy, so choose the row with rowNumber 1.

In [ ]:
query = """
 SELECT A.Product_ID, A.Customer_Hierarchy,A.Invoiced_quantity_in_Pieces
  FROM (
  SELECT 
  *,
  ROW_NUMBER() OVER(PARTITION BY Customer_Hierarchy ORDER BY Invoiced_quantity_in_Pieces DESC) rowNumber
  FROM {DATASET}.total_orders
  )A
  WHERE A.rowNumber =1;
""".format(
    DATASET=DATASET
)
query_job = client.query(query)
print(query_job.result())

In [ ]:
query_job.to_dataframe()

From the above result, you can infer the following:

- Under the **Food** category, **SKU 62** has the maximum orders.
- Under the **Manufacturing** category, **SKU 17** has the maximum orders.
- Under the **Paper** category, **SKU 107** has the maximum orders.
- Under the **Publishing** category, **SKU 8** has the maximum orders.
- Under the **Utilities** category, **SKU 140** has the maximum orders.

Given that there are too many ids and only a few records for most of them, consider only the above `Product_ID`s for which there are a maximum number of orders. 

**Note**: The `Invoiced_quantity_in_Pieces` field seems to be a *float* type rather than an *int* type as it should be. This could be because the data itself might be averaged in the first place.

#### Check the various prices available for these `Product_ID`s.

First, select rows from the `required_columns_final` view that have your desired product ID and customer hierarchy.

In [ ]:
query = """
SELECT * FROM {DATASET}.required_columns_final WHERE Product_ID="SKU 62" AND Customer_Hierarchy="Food"
""".format(
    DATASET=DATASET
)
query_job = client.query(query)
df_sku_62 = query_job.to_dataframe()
df_sku_62

Then plot various prices available for these `Product_ID`s.

In [ ]:
print(df_sku_62["List_Price_Converged"].value_counts())

In [ ]:
query = """
SELECT * FROM {DATASET}.required_columns_final WHERE Product_ID="SKU 17" AND Customer_Hierarchy="Manufacturing"
""".format(
    DATASET=DATASET
)
query_job = client.query(query)
df_sku_17 = query_job.to_dataframe()
df_sku_17

In [ ]:
print(df_sku_17["List_Price_Converged"].value_counts())

In [ ]:
query = """
SELECT * FROM {DATASET}.required_columns_final WHERE Product_ID="SKU 107" AND Customer_Hierarchy="Paper"
""".format(
    DATASET=DATASET
)
query_job = client.query(query)
df_sku_107 = query_job.to_dataframe()
df_sku_107

In [ ]:
print(df_sku_107["List_Price_Converged"].value_counts())

In [ ]:
query = """
SELECT * FROM {DATASET}.required_columns_final WHERE Product_ID="SKU 8" AND Customer_Hierarchy="Publishing"
""".format(
    DATASET=DATASET
)
query_job = client.query(query)
df_sku_8 = query_job.to_dataframe()
df_sku_8

In [ ]:
print(df_sku_8["List_Price_Converged"].value_counts())

In [ ]:
query = """
SELECT * FROM {DATASET}.required_columns_final WHERE Product_ID="SKU 140" AND Customer_Hierarchy="Utilities"
""".format(
    DATASET=DATASET
)
query_job = client.query(query)
df_sku_140 = query_job.to_dataframe()

In [ ]:
print(df_sku_140["List_Price_Converged"].value_counts())

In the publishing category, exclude `Product_ID`, `SKU 8`, and `SKU 17`, as they have fewer than or equal to two different prices in the entire dataset. Instead, consider the remaining products for building the forecast model on the timeseries data with varying prices.

#### Join the data for all the `Product_ID`s into one dataframe and remove duplicate records.

In [ ]:
df_final = pd.concat([df_sku_62, df_sku_107, df_sku_140])
df_final = (
    df_final[
        [
            "Product_ID",
            "Fiscal_Date",
            "Customer_Hierarchy",
            "List_Price_Converged",
            "Invoiced_quantity_in_Pieces",
        ]
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)
df_final

#### Save the data to a BigQuery table.

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID)

job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        bigquery.SchemaField("Product_ID", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Fiscal_Date", bigquery.enums.SqlTypeNames.DATE),
        bigquery.SchemaField("List_Price_Converged", bigquery.enums.SqlTypeNames.FLOAT),
        bigquery.SchemaField(
            "Invoiced_quantity_in_Pieces", bigquery.enums.SqlTypeNames.FLOAT
        ),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

# save the dataframe to a table in the created dataset
job = bq_client.load_table_from_dataframe(
    df_final,
    "{}.{}.{}".format(PROJECT_ID, DATASET, TRAINING_DATA_TABLE),
    job_config=job_config,
)  # Make an API request.
print(job.result())  # Wait for the job to complete.

# Train the model using BigQuery ML
<a name="section-9"></a>

Train an [Arima-Plus](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series) model on the data using BigQuery ML.

#@bigquery
create or replace model [your-dataset-id].bqml_arima
options
 (model_type = 'ARIMA_PLUS',
  time_series_timestamp_col = 'Fiscal_Date',
  time_series_data_col = 'Invoiced_quantity_in_Pieces',
  time_series_id_col = 'ID'
 ) as
select
 Fiscal_Date,
 Concat(Product_ID,"_" ,Cast(List_Price_Converged as string)) as ID,
 Invoiced_quantity_in_Pieces
from
 [your-dataset-id].TRAINING_DATA


In [ ]:
query = """
create or replace model `{PROJECT_ID}.{DATASET}.bqml_arima`
options
 (model_type = 'ARIMA_PLUS',
  time_series_timestamp_col = 'Fiscal_Date',
  time_series_data_col = 'Invoiced_quantity_in_Pieces',
  time_series_id_col = 'ID'
 ) as
select
 Fiscal_Date,
 Concat(Product_ID,"_" ,Cast(List_Price_Converged as string)) as ID,
 Invoiced_quantity_in_Pieces
from
 `{DATASET}.{TRAINING_DATA_TABLE}`""".format(
    PROJECT_ID=PROJECT_ID, DATASET=DATASET, TRAINING_DATA_TABLE=TRAINING_DATA_TABLE
)
query_job = client.query(query)
print(query_job.result())

## Generate forecasts from the model
<a name="section-10"></a>

Predict the sales for the next 30 days for each id and save to a dataframe.

In [ ]:
query = '''
DECLARE HORIZON STRING DEFAULT "30"; #number of values to forecast
DECLARE CONFIDENCE_LEVEL STRING DEFAULT "0.90"; ## required confidence level

EXECUTE IMMEDIATE format("""
    SELECT
      *
    FROM 
      ML.FORECAST(MODEL {DATASET}.bqml_arima, 
                  STRUCT(%s AS horizon, 
                         %s AS confidence_level)
                 )
    """,HORIZON,CONFIDENCE_LEVEL)'''.format(
    DATASET=DATASET
)
job = client.query(query)
dfforecast = job.to_dataframe()
dfforecast.head()

## Interpret the results to choose the best price
<a name="section-11"></a>

#### Calculate average forecast values for the forecast duration.

In [ ]:
dfforecast_avg = (
    dfforecast[["ID", "forecast_value"]].groupby("ID", as_index=False).mean()
)

#### Extract the ID and Price fields from the ID field.

In [ ]:
dfforecast_avg["Product_ID"] = dfforecast_avg["ID"].apply(lambda x: x.split("_")[0])
dfforecast_avg["Price"] = dfforecast_avg["ID"].apply(lambda x: x.split("_")[1])

#### Plot the average forecasted sales vs. the price of the product.

In [ ]:
for i in dfforecast_avg["Product_ID"].unique():
    dfforecast_avg[dfforecast_avg["Product_ID"] == i].set_index("Price").sort_values(
        "forecast_value"
    ).plot(kind="bar")
    plt.title("Price vs. Average Sales for " + i)
    plt.show()

Based on the plots for price vs. the average forecasted orders, the following prices can be recommended to maximize orders for each of the considered `Product_IDs`:

- SKU 107's price range can be from 4.44 - 4.73 units
- SKU 140's price can be 1.95 units
- SKU 62's price can be 4.23 units


## Clean Up
<a name="section-12"></a>

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial. The following code deletes the entire dataset.

In [ ]:
# Set dataset_id to the ID of the dataset to fetch.
dataset_id = "{PROJECT_ID}.{DATASET}".format(PROJECT_ID=PROJECT_ID, DATASET=DATASET)

# Use the delete_contents parameter to delete a dataset and its contents.
# Use the not_found_ok parameter to not receive an error if the dataset has already been deleted.
client.delete_dataset(
    dataset_id, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(dataset_id))